<a href="https://colab.research.google.com/github/jainsam123/hackfest-2021/blob/main/recommender_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [112]:
!pip install annoy

In [113]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import re
import os
import base64
import time
import sys
import numpy as np
import uuid
import json
import tensorflow as tf
import requests
from tensorflow.keras.models import Model
from annoy import AnnoyIndex

In [ ]:
path = "/content/drive/MyDrive/models/xception_224x224_adam_batch32_8labels_5000each_10ep_ft16ep.h5"

In [ ]:
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224

In [ ]:
model = None
feature_extractor = None
ann_index = []
ann_metadata = []
MODEL = 'xception_224x224_adam_batch32_8labels_5000each_10ep_ft16ep.h5'
MAX_TOP_K = 5 # the number of nearest neighbors returns from Annoy calculation

In [ ]:
labels = ['Cell_Phones_and_Accessories', 'Clothing_Men', 'Clothing_Women', 'Electronics', 'Home_and_Kitchen', 'Pet_Supplies', 'Shoes', 'Watches']

In [ ]:
model = tf.keras.models.load_model(path)

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 7, 7, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 16392     
Total params: 20,877,872
Trainable params: 6,804,776
Non-trainable params: 14,073,096
_________________________________________________________________


In [ ]:
layer_name = 'global_average_pooling2d_2'
feature_extractor = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)

In [ ]:
for i in range(len(labels)):
  ann_index_name = 'index_xception_224x224_adam_batch32_8labels_5000each_10ep_ft16ep_label_{}.ann'.format(i)
  path_ann_index = '/content/drive/MyDrive/models/annoy_index/label separated/' + ann_index_name
  

In [ ]:
for i in range(len(labels)):
        ann_index_name = 'index_xception_224x224_adam_batch32_8labels_5000each_10ep_ft16ep_label_{}.ann'.format(i)
        ann_metadata_name = 'metadata_xception_224x224_adam_batch32_8labels_5000each_10ep_ft16ep_label_{}.json'.format(i)

        path_ann_index = '/content/drive/MyDrive/models/annoy_index/label separated/' + ann_index_name
        path_ann_metadata = '/content/drive/MyDrive/models/annoy_index/label separated/' + ann_metadata_name

        with open(path_ann_metadata) as f:
            ann_metadata_data = json.load(f)
            ann_metadata.append(ann_metadata_data)

        ann_index_obj = AnnoyIndex(ann_metadata_data['features_length'], metric='angular')
        ann_index_obj.load(path_ann_index)
        ann_index.append(ann_index_obj)

In [ ]:
def get_neighbors(label, input_feature_vectors, max_neighbors=14):
    results = []
    
    # get the nearest neighbors of that first nearest neighbor
    ann_index_obj = ann_index[label]

    for item_id in ann_index_obj.get_nns_by_vector(input_feature_vectors, max_neighbors, search_k=10):
        # if item_id != top_1:
        results.append({
        'id': item_id,
        'asin': ann_metadata[label]['list_asin'][item_id]
        })

    print('get_neighbors label', label)

    return results

# New Section

In [ ]:
import cv2

In [ ]:
def preprocess_image(img_raw):
    predict_img_width = IMAGE_WIDTH
    predict_img_height = IMAGE_HEIGHT

    img_str = cv2.imread(img_raw)
    image=tf.convert_to_tensor(img_str, dtype=tf.float32)
    image = tf.image.resize(image, [predict_img_width, predict_img_height])
    image = (255 - image) / 255.0  # normalize to [0,1] range
    image = tf.reshape(image, (1, predict_img_width, predict_img_height, 3))

    return image

In [ ]:
img_raw = "/content/bag1.jpg"

In [ ]:
image = preprocess_image(img_raw)

In [ ]:
prediction_probs = model.predict(image)

In [ ]:
predicted_label = np.argmax(prediction_probs, axis=1)[0]

In [ ]:
input_feature_vectors = feature_extractor.predict(image)
input_feature_vectors = input_feature_vectors.flatten()

In [ ]:
input_feature_vectors = input_feature_vectors / input_feature_vectors.max()

In [ ]:
top_k = get_neighbors(predicted_label, input_feature_vectors, MAX_TOP_K)

get_neighbors label 1


In [ ]:
top_k 

[{'asin': 'B002MPCB6W', 'id': 2403},
 {'asin': 'B00B28S6B4', 'id': 4639},
 {'asin': 'B00A0U7YUE', 'id': 2093},
 {'asin': 'B004WLH6P0', 'id': 4879},
 {'asin': 'B009PVZSV6', 'id': 2033}]

In [ ]:
!wget https://drive.google.com/file/d/1KoU3DCMAtGDnqK_PTboeYyK_z53sach5/view?usp=sharing

--2021-04-30 15:11:35--  https://drive.google.com/file/d/1KoU3DCMAtGDnqK_PTboeYyK_z53sach5/view?usp=sharing
Resolving drive.google.com (drive.google.com)... 74.125.142.102, 74.125.142.138, 74.125.142.101, ...
Connecting to drive.google.com (drive.google.com)|74.125.142.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘view?usp=sharing’

view?usp=sharing        [ <=>                ]  63.02K  --.-KB/s    in 0.02s   

2021-04-30 15:11:35 (2.51 MB/s) - ‘view?usp=sharing’ saved [64536]



In [ ]:
!unzip "/content/drive/MyDrive/data-v4.zip"&> /dev/null